<a href="https://colab.research.google.com/github/Romanchenko-RS/ML/blob/main/%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D1%8F_%D0%A0%D0%BE%D0%BC%D0%B0%D0%BD%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%A0_%D0%A1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Импорт необходимых библиотек**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.sparse import hstack
from sklearn.linear_model import LinearRegression

**Загружаем данные**

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-06-19 06:21:55--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 173.194.217.138, 173.194.217.102, 173.194.217.113, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-06-19 06:21:55--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.134.132, 2607:f8b0:400c:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.134.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.03s   

2024-06-19 06:21:56 (28.7 MB/

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
replace links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

Проверим, из чего состоят датасеты

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
# Обработка и объединение жанров и тегов
movies['genres'] = movies['genres'].str.replace('|', ' ')
tags_grouped = tags.groupby('movieId')['tag'].apply(' '.join).reset_index()
movies = pd.merge(movies, tags_grouped, on='movieId', how='left')
movies['tag'] = movies['tag'].fillna('')
movies['features'] = movies['genres'] + ' ' + movies['tag']

In [ ]:
# Объединение всего в один DataFrame
data = pd.merge(ratings[['userId', 'movieId', 'rating']], movies[['movieId', 'features']], on='movieId')

In [ ]:
data.head()

,userId,movieId,rating,features
0,1,1,4.0,Adventure Animation Children Comedy Fantasy pi...
1,5,1,4.0,Adventure Animation Children Comedy Fantasy pi...
2,7,1,4.5,Adventure Animation Children Comedy Fantasy pi...
3,15,1,2.5,Adventure Animation Children Comedy Fantasy pi...
4,17,1,4.5,Adventure Animation Children Comedy Fantasy pi...


In [ ]:
# Вычисление TF-IDF для фичей, состоящих из жанров и тегов
tfidf = TfidfVectorizer(stop_words='english')
X_tfidf = tfidf.fit_transform(data['features'])

NameError: name 'data' is not defined

In [ ]:
# Для дополнения данных средними рейтингами
user_average = ratings.groupby('userId')['rating'].mean().rename('user_average').reset_index()
movie_average = ratings.groupby('movieId')['rating'].mean().rename('movie_average').reset_index()

In [ ]:
data = pd.merge(data, user_average, on='userId')
data = pd.merge(data, movie_average, on='movieId')

In [ ]:
data.head()

,userId,movieId,rating,features,user_average,movie_average
0,1,1,4.0,Adventure Animation Children Comedy Fantasy pi...,4.366379,3.92093
1,5,1,4.0,Adventure Animation Children Comedy Fantasy pi...,3.636364,3.92093
2,7,1,4.5,Adventure Animation Children Comedy Fantasy pi...,3.230263,3.92093
3,15,1,2.5,Adventure Animation Children Comedy Fantasy pi...,3.448148,3.92093
4,17,1,4.5,Adventure Animation Children Comedy Fantasy pi...,4.209524,3.92093


In [ ]:
# Отбор признаков для конечного датасета
X_tfidf = hstack([X_tfidf, data[['user_average', 'movie_average']].values])

**Обучение модели**

In [ ]:
# Обучение модели регрессии
X = X_tfidf
y = data['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Оценка модели
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)

print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')

MSE: 0.6651
RMSE: 0.8155


In [ ]:
# Создание словаря для записи результатов
results = pd.DataFrame(columns=['Датасет', 'MSE', 'RMSE'])

In [ ]:
results.loc[len(results)] = ['По средним значениям оценок', mse, rmse]

**Добавим медианные значения оценок от пользователей и оценок по фильмам**

In [ ]:
# Рассчитываем медианные оценки каждого фильма
movie_median_ratings = ratings.groupby('movieId')['rating'].median().rename('movie_median_rating').reset_index()
# Рассчитываем медианные оценки, выставленные пользователями
user_median_ratings = ratings.groupby('userId')['rating'].median().rename('user_median_rating').reset_index()

In [ ]:
# Добавляем медианные оценки пользователей и фильмов к основному DataFrame
data1 = pd.merge(data, user_median_ratings, on='userId', how='left')
data1 = pd.merge(data, movie_median_ratings, on='movieId', how='left')

In [ ]:
# Вычисление TF-IDF для фичей, состоящих из жанров и тегов
tfidf1 = TfidfVectorizer(stop_words='english')
X_tfidf1 = tfidf1.fit_transform(data1['features'])

In [ ]:
# Отбор признаков для конечного датасета
X_tfidf1 = hstack([X_tfidf1, data1[['user_average', 'movie_average']].values])

In [ ]:
# Обучение модели регрессии
X1 = X_tfidf1
y1 = data1['rating'].values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [ ]:
model1 = LinearRegression()
model1.fit(X_train1, y_train1)

LinearRegression()

In [ ]:
# Оценка модели
y_pred1 = model1.predict(X_test1)
mse1 = mean_squared_error(y_test1, y_pred1)
rmse1 = sqrt(mse1)

print(f'MSE: {mse1:.4f}')
print(f'RMSE: {rmse1:.4f}')

MSE: 0.6668
RMSE: 0.8166


In [ ]:
results.loc[len(results)] = ['С добавлением медианных значений', mse1, rmse1]

**Выводы**

In [ ]:
results

,Датасет,MSE,RMSE
0,По средним значениям оценок,0.665087,0.815529
1,С добавлением медианных значений,0.666788,0.816571


Как видно, добавление дополнительных переменных в виде медианных значений по оценкам пользователей и оценок фильмов, получаем увеличение метрики RMSE, что говорит об ухудшении качества.

Для анализа использовалась самая простая линейная модель, поэтому и значение RMSE получилось довольно большое. Возможно, стоит применить другие, более сложные методы, чтобы с большей точностью предсказывать оценки фильмов на основе жанров и тегов.